# 使用 LangChain 构建一个 RAG 应用

## RAG 是什么

RAG 是一种将检索到的文档上下文与大语言模型（LLM）结合起来生成答案的技术。

整个过程主要分为以下几个步骤：

1. 加载文档：将原始数据(来源可能是在线网站、本地文件、各类平台等)加载到 LangChain 中。
1. 文档分割：将加载的文档分割成较小的块，以适应模型的上下文窗口，并更容易进行向量嵌入和检索。
1. 存储嵌入：将分割后的文档内容嵌入到向量空间，并存储到向量数据库中，以便后续检索。
1. 检索文档：通过查询向量数据库，检索与问题最相关的文档片段。
1. 生成回答：将检索到的文档片段与用户问题组合，生成并返回答案。

通过这些步骤，可以构建一个强大的问答系统，将复杂任务分解为更小的步骤并生成详细回答。

![rag](../images/rag.png)

In [1]:
!pip install langchain langchain_community langchain_chroma

## **RAG 开发指南**

**本指南将详细介绍如何使用 LangChain 框架构建一个基于检索增强生成 (RAG) 的应用。**

下面是基于 LangChain 实现的 RAG 的核心步骤与使用到的关键代码抽象（类型、方法、库等）:

1. **加载文档**: 使用 `WebBaseLoader` 类从指定来源加载内容，并生成 `Document` 对象（依赖 `bs4` 库）。
2. **文档分割**: 使用 `RecursiveCharacterTextSplitter` 类的 `split_documents()` 方法将长文档分割成较小的块。
3. **存储嵌入**: 使用 `Chroma` 类的 `from_documents()` 方法将分割后的文档内容嵌入向量空间，并存储在向量数据库中（使用 `OpenAIEmbeddings`），并可以通过检查存储的向量数量来确认存储成功。。
4. **检索文档**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法基于查询从向量数据库中检索最相关的文档片段。
5. **生成回答**: 使用 `ChatOpenAI` 类的 `invoke()` 方法，将检索到的文档片段与用户问题结合，生成回答（通过 `RunnablePassthrough` 和 `StrOutputParser`）。

我们使用的文档是Lilian Weng撰写的《LLM Powered Autonomous Agents》博客文章（https://lilianweng.github.io/posts/2023-06-23-agent/ ），最终构建好的 RAG 应用支持我们询问关于该文章内容的相关问题。

In [1]:
# 导入必要的库
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### Step 1: 加载文档

- **描述**: 使用 `DocumentLoader` 从指定来源（如网页）加载内容，并将其转换为 `Document` 对象。
- **重要代码抽象**:
  - 类: `WebBaseLoader`
  - 方法: `load()`
  - 库: `bs4` (BeautifulSoup)
- **代码解释**:
  - **文档加载**: 使用 `WebBaseLoader` 从网页加载内容，并通过 `BeautifulSoup` 解析 HTML，提取重要的部分。
  - **检查加载数量**: 打印加载的文档数量，确保所有文档正确加载。
  - **验证文档内容**: 输出第一个文档的部分内容，确认加载的数据符合预期。

In [2]:
# 使用 WebBaseLoader 从网页加载内容，并仅保留标题、标题头和文章内容
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [3]:
type(docs)

list

In [4]:
print(len(docs))

1


In [5]:
# 检查加载的文档内容长度
print(len(docs[0].page_content))  # 打印第一个文档内容的长度

43131


In [6]:
# 查看第一个文档（前100字符）
print(docs[0].page_content[:100])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |


#### **扩展：加载pdf文件**
>**api链接：**
[LangChain PyPDFLoader API](https://python.langchain.com/v0.2/docs/integrations/document_loaders/pypdfloader)
> <br>**说明:**
> pdf文件是以页面为基础的输出格式，每一页独立，加载时通常按页处理,因此,会按照页面分割为多个 Document 对象。

In [7]:
# 安装pypdf依赖包
!pip install pypdf

In [8]:
# 导入PyPDFLoader
from langchain.document_loaders import PyPDFLoader

In [9]:
# 加载本地 PDF 文档
pdf_loader = PyPDFLoader("./testdocs/drawingtest.pdf")
pdf_documents = pdf_loader.load()

In [10]:
# 验证加载的文档,pdf文件加载后每一页默认是一个document对象
print(f"Loaded {len(pdf_documents)} documents from PDF.")
print(f"First document content: {pdf_documents[0].page_content[:500]}")  # 打印前500个字符

Loaded 30 documents from PDF.
First document content: 001
问题：你们的美术课程适合多大的孩子？
回答：我们的美术课程专为4至10岁的孩子设计，旨在为他们提供一个有趣的艺术启蒙体
验。
---
002
问题：这门课都教些什么呢？
回答：课程内容包括绘画、手工制作、色彩理论和艺术欣赏等，帮助孩子们全方位了解艺术。
---
003
问题：授课是怎么进行的？是大班还是小班？
回答：我们采用小班授课，注重互动与实践，让每个孩子都能得到充分的关注与指导。
---
004
问题：孩子上课需要带自己的材料吗？
回答：大部分材料由我们提供，但建议孩子自带一些基础工具，如水彩笔和素描本，以增强
个人创作体验。
---
005
问题：上课时间是什么时候？多久上一次课？
回答：每节课通常为1.5小时，每周上一次课，具体时间可根据家长和孩子的需求进行调整。
---
006
问题：老师很专业吗？有什么资格？
回答：我们的教师均具有专业的美术教育背景，并拥有丰富的儿童艺术教学经验。
---
007
问题：课程费用贵不贵？一般多少？
回答：课程费用根据不同班级和课程时长而定，请联系我们获取详细的价格信息。
---
008
问题：可以先试听一节课吗？要怎么预约？
回答


#### **扩展：加载word文档**
> **API链接**：[LangChain Docx2txtLoader API](https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/#using-docx2txt)
> <br> **注意**：
> DOCX 格式是基于 XML 的结构化文档，通常按照段落、表格、标题等逻辑结构进行组织，没有页面的限制，整个文件会被作为一个整体处理，因此，DOCX 文件一般会作为一个整体加载为单一 Document 对象。

In [10]:
# 安装依赖包
!pip install docx2txt

In [11]:
# 导入 `Docx2txtLoader`
from langchain_community.document_loaders import Docx2txtLoader

In [12]:
# 加载本地 DOCX 文档
docx2txt_loader = Docx2txtLoader("./testdocs/drawingtest.docx")
docx2txt_documents = docx2txt_loader.load()

In [13]:
# 验证加载的文档
print(f"Loaded {len(docx2txt_documents)} documents from DOCX.")
print(f"First document content: {docx2txt_documents[0].page_content[:500]}")  # 打印前500个字符

Loaded 1 documents from DOCX.
First document content: 001

问题：你们的美术课程适合多大的孩子？

回答：我们的美术课程专为4至10岁的孩子设计，旨在为他们提供一个有趣的艺术启蒙体验。



---

002

问题：这门课都教些什么呢？

回答：课程内容包括绘画、手工制作、色彩理论和艺术欣赏等，帮助孩子们全方位了解艺术。



---

003

问题：授课是怎么进行的？是大班还是小班？

回答：我们采用小班授课，注重互动与实践，让每个孩子都能得到充分的关注与指导。



---

004

问题：孩子上课需要带自己的材料吗？

回答：大部分材料由我们提供，但建议孩子自带一些基础工具，如水彩笔和素描本，以增强个人创作体验。



---

005

问题：上课时间是什么时候？多久上一次课？

回答：每节课通常为1.5小时，每周上一次课，具体时间可根据家长和孩子的需求进行调整。



---

006

问题：老师很专业吗？有什么资格？

回答：我们的教师均具有专业的美术教育背景，并拥有丰富的儿童艺术教学经验。



---

007

问题：课程费用贵不贵？一般多少？

回答：课程费用根据不同班级和课程时长而定，请联系我们获取详细的


### Step 2: 文档分割

- **描述**: 使用文本分割器将加载的长文档分割成较小的块，以便嵌入和检索。
- **重要代码抽象**:
  - 类: `RecursiveCharacterTextSplitter`
  - 方法: `split_documents()`
- **代码解释**:
  - **文档分割**: 使用 `RecursiveCharacterTextSplitter` 按字符大小分割文档块，设置块大小和重叠字符数，确保文档块适合模型处理。
  - **检查块数量**: 打印分割后的文档块数量，确保分割操作正确执行。
  - **验证块大小**: 输出第一个块的字符数，确认分割块的大小是否符合预期。

In [14]:
# 使用 RecursiveCharacterTextSplitter 将文档分割成块，每块1000字符，重叠200字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [38]:
type(all_splits)

list

In [15]:
# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量

66


In [16]:
print(len(all_splits[0].page_content))  # 打印第一个块的字符数

969


In [17]:
print(all_splits[0].page_content)  # 打印第一个块的内容

LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory


In [18]:
print(all_splits[0].metadata)  # 打印第一个块的元数据

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}


###  **扩展：自定义分割符进行文档分割**

In [19]:
# 导入TextSplitter
from langchain.text_splitter import TextSplitter

In [20]:
# 自定义文本分割器，按问答编号进行分割
class QATextSplitter(TextSplitter):
    def split_text(self, text):
        # 按编号 '---' 分割文档
        return text.split('---')


#### **pdf文档分割**

In [21]:
# 加载pdf文档对象列表，打印列表长度
documents = pdf_documents

print(f"documents的数据类型是:{type(documents)}\n")

print(f"documents的文档数量为:{len(documents)}")

documents的数据类型是:<class 'list'>

documents的文档数量为:30


In [22]:
# 实例化自定义的文本分割器
text_splitter = QATextSplitter()

In [23]:
qa_chunks = []

# 遍历文档列表，并对每个文档的内容进行自定义分割
for doc in documents:
    # 预处理PDF文档内容，去除多余的换行符、分页符、空白字符
    content = doc.page_content.strip()  # 去掉文档前后的空白字符
    content = content.replace("\n\n", "\n")  # 合并多余的换行符，防止额外的分割
    content = content.replace("\x0c", "")  # 去除分页符
    # 使用自定义分割器进行分割
    chunks = text_splitter.split_text(content)
    
    # 过滤掉空白块或仅包含空白字符的块
    filtered_chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    
    # 将过滤后的块添加到结果列表
    qa_chunks.extend(filtered_chunks)

In [24]:
# 检查分割后的块数量和内容
print(len(qa_chunks))  # 打印分割后的文档块数量

262


In [25]:
# 打印第一个文本块的内容长度
print(len(qa_chunks[0]))

66


In [40]:
# 打印第一个文本块的内容
print(qa_chunks[0])

001 问题：你们的美术课程适合多大的孩子？ 回答：我们的美术课程专为4至10岁的孩子设计，旨在为他们提供一个有趣的艺术启蒙体 验。


In [27]:
# 打印最后一个文本块的内容
print(qa_chunks[-1])

250
问题：你们的课程适合未来想当艺术家的孩子吗？
回答：非常适合！我们的课程为孩子们打下良好的艺术基础，也为未来的艺术之路奠定信心。


#### **Word文档分割**

In [28]:
# 加载word文档对象，打印列表长度
documents = docx2txt_documents

print(f"documents的数据类型：{type(documents)}\n")
print(f"documents的文档数量: {len(documents)}")

documents的数据类型：<class 'list'>

documents的文档数量: 1


In [29]:
# 实例化自定义的文本分割器
text_splitter = QATextSplitter()

In [30]:
qa_chunks = []

# 遍历文档列表，并对每个文档的内容进行自定义分割
for doc in documents:
    # 预处理文档内容，去除多余的换行符和空白字符
    content = doc.page_content.strip()  # 去掉文档前后的空白字符
    content = content.replace("\n\n", "\n")  # 合并多余的换行符，防止额外的分割

    # 使用自定义分割器进行分割
    chunks = text_splitter.split_text(content)
    
    # 过滤掉空白块或仅包含空白字符的块
    filtered_chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    
    # 将过滤后的块添加到结果列表
    qa_chunks.extend(filtered_chunks)

In [31]:
# 检查分割后的文档块的数量
print(len(qa_chunks))

250


In [32]:
# 打印第一个文本块的长度
print(len(qa_chunks[0]))

65


In [33]:
# 打印第一个文本块的内容
print(qa_chunks[0])

001
问题：你们的美术课程适合多大的孩子？
回答：我们的美术课程专为4至10岁的孩子设计，旨在为他们提供一个有趣的艺术启蒙体验。


In [34]:
# 打印最后一个文本块的内容
print(qa_chunks[-1])

250
问题：你们的课程适合未来想当艺术家的孩子吗？
回答：非常适合！我们的课程为孩子们打下良好的艺术基础，也为未来的艺术之路奠定信心。


####  **解决PyPDFloader解析器带来后续分割总块数与期望不符合的问题**

In [35]:
import pdfplumber

# 自定义分割器类，按 '---' 分割内容
class QATextSplitter:
    def split_text(self, text):
        # 按 '---' 分割文本
        return text.split('---')

# 加载 PDF 文档并提取文本内容
with pdfplumber.open("./testdocs/drawingtest.pdf") as pdf:
    content = ""
    for page in pdf.pages:
        content += page.extract_text() + "\n"

# 去除多余的空行和分页符，合并换行符
content = content.replace("\n\n", "\n").replace("\n", " ")  # 替换所有换行符为空格
content = content.replace("\x0c", "")  # 去除分页符

# 使用自定义文本分割器
text_splitter = QATextSplitter()

# 使用自定义分割器进行分割
chunks = text_splitter.split_text(content)

# 过滤掉空白块或仅包含空白字符的块
qa_chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

# 打印分割后的块总数和部分内容，以便检查
print(f"总块数: {len(qa_chunks)}")

# 输出分割后的块信息（打印前10个块）
for i, chunk in enumerate(qa_chunks[:10]):
    print(f"块 {i+1} 内容:\n{chunk}\n")

# 打印最后一个块
print(f"最后一个块的内容:\n{qa_chunks[-1]}")


总块数: 250
块 1 内容:
001 问题：你们的美术课程适合多大的孩子？ 回答：我们的美术课程专为4至10岁的孩子设计，旨在为他们提供一个有趣的艺术启蒙体 验。

块 2 内容:
002 问题：这门课都教些什么呢？ 回答：课程内容包括绘画、手工制作、色彩理论和艺术欣赏等，帮助孩子们全方位了解艺术。

块 3 内容:
003 问题：授课是怎么进行的？是大班还是小班？ 回答：我们采用小班授课，注重互动与实践，让每个孩子都能得到充分的关注与指导。

块 4 内容:
004 问题：孩子上课需要带自己的材料吗？ 回答：大部分材料由我们提供，但建议孩子自带一些基础工具，如水彩笔和素描本，以增强 个人创作体验。

块 5 内容:
005 问题：上课时间是什么时候？多久上一次课？ 回答：每节课通常为1.5小时，每周上一次课，具体时间可根据家长和孩子的需求进行调整。

块 6 内容:
006 问题：老师很专业吗？有什么资格？ 回答：我们的教师均具有专业的美术教育背景，并拥有丰富的儿童艺术教学经验。

块 7 内容:
007 问题：课程费用贵不贵？一般多少？ 回答：课程费用根据不同班级和课程时长而定，请联系我们获取详细的价格信息。

块 8 内容:
008 问题：可以先试听一节课吗？要怎么预约？ 回答：我们欢迎家长和孩子来参加免费的试听课，您可以提前预约。

块 9 内容:
009 问题：孩子完成课程后，可以展示自己的作品吗？ 回答：是的，我们会定期举办孩子们的作品展览，让他们有机会展示自己的创作成果。

块 10 内容:
010 问题：如果孩子缺课，有补课的机会吗？ 回答：当然可以！我们将为缺课的孩子提供补课选项，以确保他们能够跟上进度。

最后一个块的内容:
250 问题：你们的课程适合未来想当艺术家的孩子吗？ 回答：非常适合！我们的课程为孩子们打下良好的艺术基础，也为未来的艺术之路奠定信心。


### Step 3: 存储嵌入

- **描述**: 将分割后的文档内容嵌入到向量空间中，并存储到向量数据库，以便后续检索。
- **重要代码抽象**:
  - 类: `Chroma`
  - 方法: `from_documents()`
  - 类: `OpenAIEmbeddings`
- **代码解释**:
  - **存储嵌入**: 使用 `Chroma.from_documents()` 方法将所有分割的文档片段进行嵌入(`OpenAIEmbeddings`嵌入模型)，将文档片段嵌入向量空间，并存储在向量数据库中。

#### Chroma 基础使用

**下面是初始化 Chroma 数据库（仅实例化，未存储向量数据）的常见做法：**
https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/

**使用构造函数初始化**: 在本地持久化存储 Chroma 数据库.

```python
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)
```

**使用 Client 初始化**: 更方便地访问底层数据库/集合。

```python
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)
```


**我们直接使用 `Chroma.from_documents()` 方法 实例化+数据存储**:

该方法返回 Chroma 实例，数据类型为`langchain_chroma.vectorstores.Chroma`，详细 API 文档： https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/

In [36]:
# 使用 Chroma 向量存储和 OpenAIEmbeddings 模型，将分割的文档块嵌入并存储
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings()
)

In [37]:
# 查看 vectorstore 数据类型
type(vectorstore) 

langchain_chroma.vectorstores.Chroma

#### **使用FAISS**
**FAISS**：与chroma类似，都是嵌入式向量数据库，由facebook提供
<br>**API文档**: [Langchain FAISS API](https://python.langchain.com/v0.2/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)

In [47]:
# 安装faiss-cpu
!pip install faiss-cpu

In [50]:
# 使用faiss向量存储和OpenAIEmmbeddings模型将分割的文本块嵌入并存储
from langchain.vectorstores import FAISS

# 使用 FAISS.from_texts()存储
vectorstore_faiss = FAISS.from_texts(
    texts=qa_chunks,  # 使用分割后的文档块
    embedding=OpenAIEmbeddings()
)

In [51]:
# 查看 vectorstore_faiss 数据类型
type(vectorstore_faiss) 

langchain_community.vectorstores.faiss.FAISS

### Step 4: 检索文档

- **描述**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法，从向量数据库中检索与查询最相关的文档片段。
- **重要代码抽象**:
  - 类: `VectorStoreRetriever`
  - 方法: `as_retriever()`, `invoke()`
- **代码解释**:
  - **文档检索**: 将向量存储转换为检索器，并基于查询执行相似性搜索，获取相关文档片段。
  - **检查检索数量**: 打印检索到的文档片段数量，确保检索操作成功。
  - **验证检索内容**: 输出第一个检索到的文档内容，确认检索结果与预期相符。

在 LangChain 中，所有向量数据库都支持**vectorstore.as_retriever** 方法，实例化该数据库对应的检索器（Retriever），数据类型为`VectorStoreRetriever`，详细 API 文档：https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html

In [38]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [39]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [40]:
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

In [41]:
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量

6


In [42]:
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


#### **检索FAISS库**

In [52]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever_faiss = vectorstore_faiss.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [54]:
type(retriever_faiss)

langchain_core.vectorstores.VectorStoreRetriever

In [55]:
retriever_faiss_docs = retriever_faiss.invoke("包含哪些课程内容？")

In [57]:
# 检查检索到的文本内容
print(len(retriever_faiss_docs))

6


In [59]:
print(retriever_faiss_docs[0].page_content)

002 问题：这门课都教些什么呢？ 回答：课程内容包括绘画、手工制作、色彩理论和艺术欣赏等，帮助孩子们全方位了解艺术。


### Step 5: 生成回答

- **描述**: 将之前构建的组件（检索器、提示、LLM等）组合成一个完整的链条，实现用户问题的检索与生成回答。完整链条：输入用户问题，检索相关文档，构建提示，将其传递给模型（使用`ChatOpenAI` 类的 `invoke()` 方法），并解析输出生成最终回答。
- **重要代码抽象**:
  - 类: `ChatOpenAI`
  - 方法: `invoke()`
  - 类: `RunnablePassthrough`
  - 类: `StrOutputParser`
  - 模块：`hub`
- **代码解释**:
  - **模型初始化**: 使用 `ChatOpenAI` 类初始化一个 `GPT-4o-mini` 模型，准备处理生成任务。
  - **文档格式化**: 定义 `format_docs` 函数，用于将检索到的文档内容格式化为字符串。
  - **构建 RAG 链**: 使用 LCEL (LangChain Execution Layer) 的 `|` 操作符将各个组件连接成一个链条，包括文档检索、提示构建、模型调用以及输出解析。
  - **生成回答**: 使用 `stream()` 方法逐步输出生成的回答，并实时展示，确保生成的结果符合预期。

![retrieval](../images/retrieval.png)

#### LangChain Hub

`LangChain Hub` (https://smith.langchain.com/hub) 是一个提示词模板开源社区，为开发者提供了大量开箱即用的提示词模板。属于 `LangSmith` 产品的一部分。

下面我们尝试使用 RAG 应用的提示词模板：https://smith.langchain.com/hub/rlm/rag-prompt


```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

In [61]:
# 定义 RAG 链，将用户问题与检索到的文档结合并生成答案
llm = ChatOpenAI(model="gpt-4o-mini")

In [63]:
# 使用 hub 模块拉取 rag 提示词模板
prompt = hub.pull("rlm/rag-prompt")

D:\anacondaInstall\envs\langchain\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [64]:
# 打印模板
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [65]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [66]:
# 查看提示词
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


#### ⭐️**LCEL 在 RAG 中的应用**⭐️

##### **LCEL 概述**

LCEL 是 LangChain 中的一个重要概念，它提供了一种统一的接口，允许不同的组件（如 `retriever`, `prompt`, `llm` 等）可以通过统一的 `Runnable` 接口连接起来。每个 `Runnable` 组件都实现了相同的方法，如 `.invoke()`、`.stream()` 或 `.batch()`，这使得它们可以通过 `|` 操作符轻松连接。

##### **LCEL 中处理的组件**

- **Retriever**: 负责根据用户问题检索相关文档。
- **Prompt**: 根据检索到的文档构建提示，供模型生成回答。
- **LLM**: 接收提示并生成最终的回答。
- **StrOutputParser**: 解析 LLM 的输出，只提取字符串内容，供最终显示。

##### **LCEL 运作机制**

- **构建链条**: 通过 `|` 操作符，我们可以将多个 `Runnable` 组件连接成一个 `RunnableSequence`。LangChain 会自动将一些对象转换为 `Runnable`，如将 `format_docs` 转换为 `RunnableLambda`，将包含 `"context"` 和 `"question"` 键的字典转换为 `RunnableParallel`。

- **数据流动**: 用户输入的问题会在 `RunnableSequence` 中依次经过各个 `Runnable` 组件。首先，问题会通过 `retriever` 检索相关文档，然后通过 `format_docs` 将这些文档转换为字符串。`RunnablePassthrough` 则直接传递原始问题。最后，这些数据被传递给 `prompt` 来生成完整的提示，供 LLM 使用。

##### **LCEL 中的关键操作**

- **格式化文档**: `retriever | format_docs` 将问题传递给 `retriever` 生成文档对象，然后通过 `format_docs` 将这些文档格式化为字符串。
- **传递问题**: `RunnablePassthrough()` 直接传递原始问题，保持原样。
- **构建提示**: `{"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt` 构建完整的提示。
- **运行模型**: `prompt | llm | StrOutputParser()` 运行 LLM 生成回答，并解析输出。

#### 使用 LCEL 构建 RAG Chain

下面我们将 LCEL 的概念与代码实现结合起来，展示了如何通过一系列 `Runnable` 组件来实现完整的 RAG 流程。通过 LCEL，LangChain 提供了高度模块化和可扩展的开发方式，使复杂任务的实现变得更加简单和高效。


In [67]:
# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [49]:
# 使用 LCEL 构建 RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [50]:
# 流式生成回答
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task decomposition is the process of breaking down complex tasks into smaller, more manageable steps. It enhances model performance by allowing systems to "think step by step," transforming large tasks into simpler sub-tasks. Techniques like Chain of Thought (CoT) and Tree of Thoughts facilitate this by exploring multiple reasoning possibilities at each step.

In [51]:
# 流式生成回答
for chunk in rag_chain.stream("What is ToT?"):
    print(chunk, end="", flush=True)

ToT stands for Tree of Thoughts, which is an extension of the Chain of Thought (CoT) prompting technique. It involves decomposing a problem into multiple reasoning steps and generating various thoughts for each step, structured like a tree. The reasoning process can utilize either breadth-first search (BFS) or depth-first search (DFS) to evaluate different states.

### **提出3个问题，测试本地文档召回情况**

In [68]:
# 定义格式化文档的函数
def format_docs_faiss(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [70]:
# 使用LCEL 构建 RAG CHAIN
rag_chain_faiss = (
    {"context":retriever_faiss | format_docs_faiss, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [71]:
# 流式生成回答
for chunk in rag_chain_faiss.stream("课程怎么收费？"):
    print(chunk,end="",flush=True)

课程费用根据班级和时长不同，请联系我们获取详细价格信息。我们会不定期推出优惠活动，欢迎关注我们的官网或社交媒体以获取最新信息。

In [73]:
for chunk in rag_chain_faiss.stream("课程都包含哪些内容？"):
    print(chunk,end="",flush=True)

课程内容包括绘画、手工制作、色彩理论和艺术欣赏等，同时也融入一些艺术史知识和跨学科的内容，如科学和历史，帮助孩子全面了解艺术及其背景。

In [74]:
for chunk in rag_chain_faiss.stream("孩子学不会怎么办"):
    print(chunk,end="",flush=True)

如果孩子学不会，可以通过积极的反馈和鼓励来帮助他们，帮助他们认识到每次尝试都是成长。提供个别指导也是一个有效的方法，以确保孩子能得到适合自己的教学。重要的是让孩子明白结果并不是最重要的。

### **自定义提示词模版**

In [77]:
from langchain_core.prompts import PromptTemplate

# 自定义提示词模版
custom_template ="""
请仅根据以下内容回答最后的问题。
不要使用任何未提到的额外信息，如果找不到答案，请直接说不知道，不要尝试推测或编造答案。
答案最多使用三句话，并保持简洁明了。
请在回答结束时加上“谢谢提问！”。

{context}

问题：{question}

基于以上信息的答案：
"""
custom_rag_prompt = PromptTemplate.from_template(custom_template)

In [78]:
# # 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question"}).text)


请仅根据以下内容回答最后的问题。
不要使用任何未提到的额外信息，如果找不到答案，请直接说不知道，不要尝试推测或编造答案。
答案最多使用三句话，并保持简洁明了。
请在回答结束时加上“谢谢提问！”。

filler context

问题：filler question

基于以上信息的答案：



In [80]:
# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever_faiss | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [81]:
# 使用自定义prompt生成回答
for chunk in custom_rag_chain.stream("每周上几次课"):
    print(chunk,end="",flush=True)

每周上一次课。谢谢提问！

In [82]:
for chunk in custom_rag_chain.stream("中途可以退费么"):
    print(chunk,end="",flush=True)

不知道。谢谢提问！

In [83]:
for chunk in custom_rag_chain.stream("是否会讲述大师级的作品"):
    print(chunk,end="",flush=True)

会的，课程中会介绍如毕加索、达利、伦勃朗等艺术家的作品，帮助孩子们拓宽视野。谢谢提问！

# Homework
1. 使用其他的线上文档或离线文件，重新构建向量数据库，尝试提出3个相关问题，测试 LCEL 构建的 RAG Chain 是否能成功召回。
2. 重新设计或在 LangChain Hub 上找一个可用的 RAG 提示词模板，测试对比两者的召回率和生成质量。

### 自定义 Prompt 的示例

In [52]:
from langchain_core.prompts import PromptTemplate

# 自定义提示词模板
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [53]:
# 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question"}).text)

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

filler context

Question: filler question

Helpful Answer:


In [54]:
# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [55]:
# 使用自定义 prompt 生成回答
custom_rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complicated task into smaller, manageable steps to facilitate better understanding and execution. Techniques like Chain of Thought (CoT) and Tree of Thoughts enhance model performance by structuring reasoning and exploring multiple possibilities at each step. This allows for more efficient planning and execution of complex tasks. Thanks for asking!'